<a href="https://colab.research.google.com/github/SnipeHunting/MQ135-Air-Quality-with-ST7735-Display/blob/master/ModernPortfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [177]:
#import sys
#!{sys.executable} -m pip install yfinance --upgrade --no-cache-dir

#!{sys.executable} -m pip install fix_yahoo_finance

#!{sys.executable} -m pip install pandas_datareader --upgrade

#!{sys.executable} -m pip install numpy --upgrade

import pandas as pd

import numpy as np
import scipy as sc
from scipy import optimize
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf
import plotly.graph_objects as go

yf.pdr_override()

In [178]:
# Import data
def getData(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']

    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix


stockList = ['CVX', 'AOS', 'MCD', 'INTC', 'F', 'MSFT']
stocks = [stock+'' for stock in stockList] # for none US stocks

endDate = dt.datetime.now()
startDate = endDate-dt.timedelta(days=800)


In [179]:
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(
            np.dot(weights.T,np.dot(covMatrix, weights))
           )*np.sqrt(252) # W transpose * sigma of W
    return returns, std

In [180]:
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0.05):  #negative sharp ratio.   Need to update the riskFreeInrerest
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate)/pStd  # will give the negative sharp ratio

def maxSR(meanReturns, covMatrix, riskFreeRate = 0.05, constraintSet=(0,1)):  #constraint for now is between 0 and 1
    "Minimize the negative SR, by altering the weights of the portfolio"
    numAssets = len(meanReturns) # checks the number of stocks 
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # checking that all the summation are equal to 1
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(negativeSR, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints) # sequantial least square .  First guess is making all weigths equal
    return result

#  weights = np.array([0.3,0.3,0.4])

meanReturns, covMatrix = getData(stocks,start=startDate, end=endDate)
#  returns, std = portfolioPerformance(weights, meanReturns,covMatrix)

result = maxSR(meanReturns, covMatrix)
#maxSR,maxWeights = result['fun'], result['x']

#print(maxSR,maxWeights) # maximize sharp ratio by changing the assets allocations (trade off between return and volatility)

[*********************100%***********************]  6 of 6 completed


#Minimium Portfolio Variance

In [181]:
def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)):
    """Minimize the portfolio variance by altering the 
     weights/allocation of assets in the portfolio"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

minVarResult = minimizeVariance(meanReturns, covMatrix)
minVar,minVarWeights = minVarResult['fun'], minVarResult['x']
print(minVar,minVarWeights) # provides the allocation for minimum variablity portfolio

0.1591593465990885 [0.0814815  0.19161491 0.         0.         0.6681072  0.05879639]


#Creating the Efficient Frontier

In [182]:
def portfolioReturn(weights, meanReturns, covMatrix):
        return portfolioPerformance(weights, meanReturns, covMatrix)[0]


def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """For each returnTarget, we want to optimise the portfolio for min variance"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)

    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method = 'SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

##Use a return function that could be called by a graphing package like Plotly/Dash combination.

In [183]:
def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Read in mean, cov matrix, and other financial information
        Output, Max SR , Min Volatility, efficient frontier """
    # Max Sharpe Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)

    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation] # to round decimal values
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)

    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20) # define the number of points in the curve
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns
  
print(calculatedResults(meanReturns, covMatrix))


(28.32, 22.9,       allocation
AOS          0.0
CVX         69.0
F            6.0
INTC         0.0
MCD         24.0
MSFT         0.0, 15.55, 15.92,       allocation
AOS          8.0
CVX         19.0
F            0.0
INTC         0.0
MCD         67.0
MSFT         6.0, [0.15915907414811892, 0.1593598791532806, 0.15997898661539084, 0.16101114258555727, 0.16244864657091057, 0.164280519831082, 0.1664989063540411, 0.16916400813034463, 0.17227827726753264, 0.1757891175043732, 0.17966649199259938, 0.18391573396390318, 0.18853067862654652, 0.19348515990328996, 0.19875427318531513, 0.20431267589227908, 0.2101375843434471, 0.2162085283480526, 0.22250397673457895, 0.2290064379025881], array([0.15547141, 0.16219601, 0.16892061, 0.17564522, 0.18236982,
       0.18909442, 0.19581903, 0.20254363, 0.20926823, 0.21599284,
       0.22271744, 0.22944204, 0.23616665, 0.24289125, 0.24961585,
       0.25634046, 0.26306506, 0.26978966, 0.27651426, 0.28323887]))


#Visualising the Efficient Frontier

In [184]:
def EF_graph(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Return a graph ploting the min vol, max sr and efficient frontier"""
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet)

    #Max SR
    MaxSharpeRatio = go.Scatter(
        name='Maximium Sharpe Ratio',
        mode='markers',
        x=[maxSR_std],
        y=[maxSR_returns],
        marker=dict(color='red',size=14,line=dict(width=3, color='black'))
    )

    #Min Vol
    MinVol = go.Scatter(
        name='Mininium Volatility',
        mode='markers',
        x=[minVol_std],
        y=[minVol_returns],
        marker=dict(color='green',size=14,line=dict(width=3, color='black'))
    )

    #Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std*100, 2) for ef_std in efficientList],
        y=[round(target*100, 2) for target in targetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )

    data = [MaxSharpeRatio, MinVol, EF_curve]

    layout = go.Layout(
        title = 'Portfolio Optimisation with the Efficient Frontier',
        yaxis = dict(title='Annualised Return (%)'),
        xaxis = dict(title='Annualised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=800,
        height=600)
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

In [185]:
EF_graph(meanReturns, covMatrix)